In [ ]:
import geopandas as gpd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fiona as fio
import shapely as shp  

import esda



import geojson as gs

import seaborn as sns
import splot
from splot.esda import plot_moran
import contextily




In [ ]:
# load in geojson files

gdf_ar = gpd.read_file('data/gis/point/ar_samples_w_geol.geojson')

gdf_geol = gpd.read_file('data/gis/polygon/geology.geojson')

gdf_cnty = gpd.read_file('data/gis/polygon/county.geojson')


In [ ]:
gdf_cnty.crs

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 20), dpi=300)
gdf_ar.plot(
    column="group_five",
    cmap="viridis",
    edgecolor="white",
    linewidth=0.0,
    alpha=0.75,
    ax=ax,
)

gdf_geol.plot(
    color="none",
    edgecolor="black",
    linewidth=0.5,
    ax=ax
)

contextily.add_basemap(
    ax,
    crs=gdf_ar.crs,
    source=contextily.providers.Stamen.TerrainBackground,
)
ax.set_axis_off()

## Calculate Autocovariate 

minimum Euclidean distance (bandwidth) at which no private well had zero neighbors was 1976 meters and used this value (𝑑𝑖𝑗) in the analysis.

# Feature Engineering

Convert elevated arsenic, bedrock type, geologic belt, well depth into dummy variables for modeling. Check for interaction of belt and rock type to guage need for interaction - newly engineered crosstab variables

In [62]:
# Check category levels reflect original study
gdf_ar['belt2'].value_counts()

belt2
Kings Mountain Belt    319
Charlotte Belt         278
Inner Piedmont         129
Name: count, dtype: int64

In [63]:
gdf_ar['type'].value_counts()

type
Intrusive Rocks      400
Metamorphic Rocks    326
Name: count, dtype: int64

In [64]:
# cross tabulate belt2 and type

pd.crosstab(gdf_ar['belt2'], gdf_ar['type'])

type,Intrusive Rocks,Metamorphic Rocks
belt2,,
Charlotte Belt,231,47
Inner Piedmont,31,98
Kings Mountain Belt,138,181


In [65]:
# cross tabulate belt2, type, and geocode

pd.crosstab([gdf_ar['belt2'], gdf_ar['type']], gdf_ar['geocode'])

geocode                                CZab  CZbg  CZbl  CZfv  CZg  CZms  DOg   
belt2               type                                                        
Charlotte Belt      Intrusive Rocks       0     0     0     0   27     0   36  \
                    Metamorphic Rocks     0     0     0    47    0     0    0   
Inner Piedmont      Intrusive Rocks       0     0     0     0    0     0    0   
                    Metamorphic Rocks     5     3     0     0    0    90    0   
Kings Mountain Belt Intrusive Rocks       0     0     0     0    0     0    0   
                    Metamorphic Rocks     0     0    59     0    0     0    0   

geocode                                Mc  OCg  PPmg  PzZq  Zbt  
belt2               type                                         
Charlotte Belt      Intrusive Rocks     0    0     0   168    0  
                    Metamorphic Rocks   0    0     0     0    0  
Inner Piedmont      Intrusive Rocks    28    3     0     0    0  
                    Metamorphic Rocks   0    0     0     0    0  
Kings Mountain Belt Intrusive Rocks     2    0   128     8    0  
                    Metamorphic Rocks   0    0     0     0  122


Simplify belt and rock type values

In [66]:
# Simplify belt and rock type values
# for belt2, Charlotte Belt = CB, "Inner Piedmont" = IP, "Kings Mountain Belt" = KM
# for type Intrusive Rocks = IR, Metamorphic Rocks = MR

gdf_ar['belt2'] = gdf_ar['belt2'].replace(['Charlotte Belt', 'Inner Piedmont', 'Kings Mountain Belt'], ['CB', 'IP', 'KM'])
gdf_ar['type'] = gdf_ar['type'].replace(['Intrusive Rocks', 'Metamorphic Rocks'], ['IR', 'MR'])



Crossing belt with rock type would result in 6 columns, whereas the formation code provides more granularity with 12 columns

In [67]:
# create new variable combining belt and rock type

gdf_ar['belt_type'] = gdf_ar['belt2'] + '_' + gdf_ar['type']

gdf_ar['belt_type'].value_counts()

belt_type
CB_IR    231
KM_MR    181
KM_IR    138
IP_MR     98
CB_MR     47
IP_IR     31
Name: count, dtype: int64

---

Delete everything below once proper data with full well depth and pH is obtained

In [68]:
# if depth is missing, replace with 0 for now

gdf_ar['depth'].fillna(200, inplace=True)

# if pH is missing or equal to 0, replace with 7 for now

gdf_ar['ph'].fillna(7, inplace=True)

gdf_ar['ph'].replace(0, 7, inplace=True)

---

In [69]:
# Create categorical variable for well depth, per the study, where depth categories are <150, 150-300, and 300+

gdf_ar['depth_cat'] = pd.cut(gdf_ar['depth'], bins=[0, 150, 300, 1000], labels=['<150', '150-300', '300+'])

In [70]:
# create dummy variables for belt_type, geocode, depth_cat, and group

gdf_ar = pd.get_dummies(gdf_ar, columns=['belt_type', 'geocode', 'depth_cat', 'group'])



In [71]:
gdf_ar

,id,full_add,year_built,date_sampled,year_sampled,X,Y,ar,group_five,depth,...,geocode_Mc,geocode_OCg,geocode_PPmg,geocode_PzZq,geocode_Zbt,depth_cat_<150,depth_cat_150-300,depth_cat_300+,group_0.0,group_1.0
0,159,"152 HALL RD, GASTONIA, NC 28056",1989.0,2019-09-22,2019.0,-81.102961,35.306266,0.0,0.0,265.0,...,False,False,False,False,True,False,True,False,True,False
1,163,"105 FALLING LEAF LN, MT HOLLY, NC 28120",1989.0,2019-09-08,2019.0,-81.068957,35.305292,0.0,0.0,67.0,...,False,False,False,False,False,True,False,False,True,False
2,182,"211 WILLDON CT, GASTONIA, NC 28056",1989.0,2019-01-14,2019.0,-81.088280,35.320927,0.0,0.0,225.0,...,False,False,False,False,True,False,True,False,True,False
3,204,"1015 ELIZABETH DR, DALLAS, NC 28034",1989.0,2011-12-19,2011.0,-81.240234,35.356966,0.0,0.0,250.0,...,False,False,False,False,False,False,True,False,True,False
4,206,"529 ANTHONY GROVE RD, CROUSE, NC 28033",1989.0,2014-06-12,2014.0,-81.321470,35.386198,0.0,0.0,110.0,...,True,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,12763,"2613 LAKEFRONT DR, BELMONT, NC 28012",2018.0,2019-07-03,2019.0,-81.047580,35.182662,0.0,0.0,390.0,...,False,False,False,True,False,False,False,True,True,False
722,12765,"520 WILSON FARM RD, GASTONIA, NC 28056",2018.0,2018-07-25,2018.0,-81.114716,35.154375,0.0,0.0,200.0,...,False,False,False,False,False,False,True,False,True,False
723,12771,"2605 LAKEFRONT DR, BELMONT, NC 28012",2018.0,2019-02-19,2019.0,-81.048192,35.182678,0.0,0.0,200.0,...,False,False,False,True,False,False,True,False,True,False
724,12795,"1615 KELLY RD, MT HOLLY, NC 28120",2018.0,2019-08-08,2019.0,-81.086012,35.334429,0.0,0.0,260.0,...,False,False,False,False,True,False,True,False,True,False
